In [147]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
from google.colab import drive

In [148]:
drive.mount('/content/drive')
df = pd.read_csv('drive/My Drive/Dataset/fake_news_train.csv')
df.head(5)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [149]:
#Drop Nan values
df = df.dropna()

In [150]:
# Get the independent features
X = df.drop('label',axis = 1)

In [151]:
# Get dependent features
y = df['label']

In [152]:
y.value_counts()

0    10361
1     7924
Name: label, dtype: int64

In [153]:
X.shape

(18285, 4)

In [154]:
y.shape

(18285,)

In [155]:
import tensorflow as tf
tf.__version__

'2.8.2'

In [156]:
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Bidirectional
from tensorflow.keras.layers import Dropout

In [157]:
# Vocabulary size
voc_size = 5000

**Onehot Representation**

In [158]:
messages = X.copy()

In [159]:
messages['title'][1]

'FLYNN: Hillary Clinton, Big Woman on Campus - Breitbart'

In [160]:
messages.reset_index(inplace = True)

In [161]:
import nltk
import re
from nltk.corpus import stopwords

In [162]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [163]:
# dataset Preprocessing
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()
corpus = []
for i in range(0, len(messages)):
  review = re.sub('[^a-zA-Z]', '', messages['title'][i])
  review = review.lower()
  review = review.split()

  review = [ps.stem(word) for word in review if not word in stopwords.words('english')]
  review = ' '.join(review)
  corpus.append(review)

**Embedding Representing**

In [164]:
onehot_rep = [one_hot(words, voc_size) for words in corpus]

In [165]:
sent_length = 20
embedded_docs = pad_sequences(onehot_rep, padding = 'pre', maxlen = sent_length)

In [166]:
embedded_docs[0]

array([  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0, 524], dtype=int32)

In [167]:
# Creating Model
embedding_vector_features = 40
model = Sequential()
model.add(Embedding(voc_size, embedding_vector_features, input_length= sent_length))
model.add(LSTM(100))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer ='adam',metrics = ['accuracy'])
model.summary()

Model: "sequential_10"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_10 (Embedding)    (None, 20, 40)            200000    
                                                                 
 lstm_9 (LSTM)               (None, 100)               56400     
                                                                 
 dense_9 (Dense)             (None, 1)                 101       
                                                                 
Total params: 256,501
Trainable params: 256,501
Non-trainable params: 0
_________________________________________________________________


In [168]:
# Creating Model
embedding_vector_features = 40
model1 = Sequential()
model1.add(Embedding(voc_size,embedding_vector_features,input_length = sent_length))
model1.add(Bidirectional(LSTM(100)))
model1.add(Dropout(0.3))
model1.add(Dense(1, activation = 'sigmoid'))
model1.compile(loss='binary_crossentropy',optimizer = 'adam',metrics = ['accuracy'])
model1.summary()

Model: "sequential_11"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_11 (Embedding)    (None, 20, 40)            200000    
                                                                 
 bidirectional_5 (Bidirectio  (None, 200)              112800    
 nal)                                                            
                                                                 
 dropout_5 (Dropout)         (None, 200)               0         
                                                                 
 dense_10 (Dense)            (None, 1)                 201       
                                                                 
Total params: 313,001
Trainable params: 313,001
Non-trainable params: 0
_________________________________________________________________


In [169]:
len(embedded_docs), y.shape

(18285, (18285,))

In [170]:
import numpy as np
X_final = np.array(embedded_docs)
y_final = np.array(y)

In [171]:
X_final.shape, y_final.shape

((18285, 20), (18285,))

In [172]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(X_final,y_final,test_size=0.3,random_state = 42)

**Model Training**

In [173]:
# Final Training
#model.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=10,batch_size=64)

In [174]:
# Model Training
model1.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=10,batch_size=64)

Epoch 1/10
200/200 [==============================] - 20s 74ms/step - loss: 0.6850 - accuracy: 0.5657 - val_loss: 0.6847 - val_accuracy: 0.5664
Epoch 2/10
200/200 [==============================] - 14s 70ms/step - loss: 0.6594 - accuracy: 0.5962 - val_loss: 0.7070 - val_accuracy: 0.5366
Epoch 3/10
200/200 [==============================] - 13s 66ms/step - loss: 0.5663 - accuracy: 0.6919 - val_loss: 0.7846 - val_accuracy: 0.5334
Epoch 4/10
200/200 [==============================] - 13s 67ms/step - loss: 0.5255 - accuracy: 0.7038 - val_loss: 0.8166 - val_accuracy: 0.5290
Epoch 5/10
200/200 [==============================] - 13s 66ms/step - loss: 0.5064 - accuracy: 0.7086 - val_loss: 0.8816 - val_accuracy: 0.5261
Epoch 6/10
200/200 [==============================] - 14s 68ms/step - loss: 0.4954 - accuracy: 0.7058 - val_loss: 0.9105 - val_accuracy: 0.5312
Epoch 7/10
200/200 [==============================] - 14s 68ms/step - loss: 0.4856 - accuracy: 0.7069 - val_loss: 0.9782 - val_accuracy:

In [175]:
y_pred1 = (model1.predict(X_test) > 0.5).astype("int32")

In [176]:
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test,y_pred1)

array([[2195,  912],
       [1567,  812]])

In [177]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test,y_pred1)

0.548122493620124

In [178]:
from sklearn.metrics import classification_report
print(classification_report(y_test,y_pred1))

              precision    recall  f1-score   support

           0       0.58      0.71      0.64      3107
           1       0.47      0.34      0.40      2379

    accuracy                           0.55      5486
   macro avg       0.53      0.52      0.52      5486
weighted avg       0.53      0.55      0.53      5486

